In [1]:
import requests
from bs4 import BeautifulSoup
import time
import json



In [2]:
# Constants
base = "https://www.zillow.com"
houses = []


In [7]:
def new_details(d, h):
    url = h['Property']['HdpUrl']
    full_address = url.split('/')[-2]
    h['Property']['Address'] = full_address.replace('-', ' ')
    h['Property']['MapsUrl'] = f"https://maps.google.com/?q={full_address}"
    h['Property']['FullUrl'] = f"{base}{url}"

    for i, v in enumerate(h['Property']['PriceHistory']):
        if v['Event'] == "Listed for sale" and h['Property']['ListPrice'] == 0:
            h['Property']['ListPrice'] = v['Price']
            h['Property']['ListDate'] = v['Date']
            h['Property']['PriceHistory'] = h['Property']['PriceHistory'][i:]

        # Properties listed for sale aren't sold...
        if d['Sold']:
            if v['Event'] == "Sold" and h['Property']['SoldPrice'] == 0:
                h['Property']['SoldPrice'] = v['Price']
                h['Property']['SoldDate'] = v['Date']
                h['Property']['PriceHistory'] = h['Property']['PriceHistory'][i:]

            if h['Property']['ListPrice'] != 0 and h['Property']['SoldPrice'] != 0:
                h['Property']['PriceDiff'] = h['Property']['SoldPrice'] - h['Property']['ListPrice']
                h['Property']['PriceDiffPercent'] = int((h['Property']['SoldPrice'] / h['Property']['ListPrice']) * 100)

    return h

def filter(d, h):
    if d['Baths'] > h['Property']['ResoFacts']['Bathrooms']:
        return False

    if d['Beds'] > h['Property']['ResoFacts']['Bedrooms']:
        return False

    if d['PropertyType'] and d['PropertyType'] != h['Property']['HomeType']:
        return False

    if h['Property']['SoldPrice'] != 0 and d['Price'] > h['Property']['SoldPrice']:
        return False

    if h['Property']['ListPrice'] != 0 and d['Price'] > h['Property']['ListPrice']:
        return False

    return True

def query(d):
    status = "for_sale/"
    if d['Sold']:
        status = "recently_sold/"
    
    url = f"{base}/homes/{status}{d['Location']}_rb/"

    # Visit each page and scrape listings
    r = requests.get(url)
    soup = BeautifulSoup(r.text, 'html.parser')

    for house_link in soup.select("ul[class*='photo-cards_extra-attribution'] a[data-test='property-card-link']"):
        p_url = house_link['href']
        house_r = requests.get(p_url)
        house_soup = BeautifulSoup(house_r.text, 'html.parser')

        for script in house_soup.find_all('script'):
            if '__NEXT_DATA__' in script.string:
                data = json.loads(script.string.split('__NEXT_DATA__ = ')[1])
                h = new_details(d, data)
                if filter(d, h):
                    houses.append(h)

    if len(houses) == 0:
        print("Property search returned no results")

    return houses


In [8]:
if __name__ == "__main__":
    details = {
        'Location': '93644',
        'Price': 50000000,
        'Baths': 3,
        'Beds': 3,
        'PropertyType': '',
        'Sold': False,
    }
    houses = query(details)
    print(houses)


<Response [403]>
Property search returned no results
[]


In [11]:
import selenium
from selenium import webdriver as wb
from selenium.webdriver.support.ui import Select
import pandas as pd
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
import time
import json
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service as ChromeService
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
import requests
from bs4 import BeautifulSoup
import random
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service as ChromeService
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
from lxml.html import fromstring  
from itertools import cycle
import traceback
import random
import time
from seleniumwire import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager


In [12]:
headers = {
            'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9',
            'User-Agent': 'Chrome/102.0.5005.63',
            'Accept-Language': 'en-US,en;q=0.9',
            'Accept-Encoding': 'gzip, deflate',
            'Connection': 'keep-alive',
            'Dnt': '1',
            'Sec-Gpc': '1',
            'Upgrade-Insecure-Requests': '1'
        }

In [13]:
def get_links(driver,url,listOflinks):
    # Create a Service object with the path to chromedriver
#     service = Service(ChromeDriverManager().install())

#     # Initialize Chrome WebDriver with the service
#     webD = webdriver.Chrome(service=service)
    
#     webD.get(url)
    driver.get(url)
    wait = WebDriverWait(driver, random.uniform(5, 15))
    time.sleep(random.uniform(5, 15))
    
#     listOflinks = []
    condition = True

    while condition:
        time.sleep(3)

        # Find all <a> elements with the data-test attribute 'property-card-link'
        propertyLinks = driver.find_elements(By.XPATH, "//a[@data-test='property-card-link']")

        for el in propertyLinks:
            href = el.get_attribute('href')
            if href:
                listOflinks.append(href)

        # Attempt to find and click the 'next' button for pagination (if exists)
        try:
            next_button = driver.find_element(By.CLASS_NAME, 'a-last').find_element(By.TAG_NAME, 'a')
            next_button.click()
        except Exception as e:
            print(f"No more pages or error navigating to next page: {e}")
            condition = False

    # Close the browser when done
#     .quit()
    return listOflinks


In [14]:
# page_num=1
location="oakhurst-ca"
# # url=f"https://www.zillow.com/homes/for_sale/{page_num}_p/?searchQueryState=%7B%22pagination%22%3A%7B%22currentPage%22%3A{page_num}%7D%2C%22isMapVisible%22%3Atrue%2C%22mapBounds%22%3A%7B%22west%22%3A-86.54695855957031%2C%22east%22%3A-86.18303644042969%2C%22south%22%3A39.465661316236485%2C%22north%22%3A39.68583094486843%7D%2C%22mapZoom%22%3A11%2C%22filterState%22%3A%7B%22sort%22%3A%7B%22value%22%3A%22globalrelevanceex%22%7D%2C%22ah%22%3A%7B%22value%22%3Atrue%7D%7D%2C%22isListVisible%22%3Atrue%7D"
# url=f"https://www.zillow.com/{location}/{page_num}_p/?searchQueryState=%7B%22isMapVisible%22%3Atrue%2C%22mapBounds%22%3A%7B%22north%22%3A37.636266154806954%2C%22south%22%3A37.18248513011958%2C%22east%22%3A-119.24884988085937%2C%22west%22%3A-119.97669411914062%7D%2C%22filterState%22%3A%7B%22sort%22%3A%7B%22value%22%3A%22globalrelevanceex%22%7D%7D%2C%22isListVisible%22%3Atrue%2C%22regionSelection%22%3A%5B%7B%22regionId%22%3A13070%2C%22regionType%22%3A6%7D%5D%2C%22pagination%22%3A%7B%22currentPage%22%3A{page_num}%7D%7D"

In [ ]:
user_agents = ["Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/119.0.0.0 Safari/537.36"]

# Set the desired headers
custom_headers = {
    "accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9",
    "accept-language": "en-GB;q=0.9,en-US;q=0.8,en;q=0.7",
    "dpr": "1",
    "sec-fetch-dest": "document",
    "sec-fetch-mode": "navigate",
    "sec-fetch-site": "none",
    "sec-fetch-user": "?1",
    "upgrade-insecure-requests": "1",
    "user-agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/119.0.0.0 Safari/537.36",
    "usePrimedCacheWhenDisabled": "true"
}

ans = []

for page_num in range(1, 5):
    user_agent = random.choice(user_agents)
    options = Options()
    options.add_argument('--start-maximized')
    
    # Initialize WebDriver with the current options
    driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()), options=options)
    
    # Set the custom headers for the driver
    driver.header_overrides = custom_headers
    
    url = f"https://www.zillow.com/{location}/{page_num}_p/?searchQueryState=%7B%22isMapVisible%22%3Atrue%2C%22mapBounds%22%3A%7B%22north%22%3A37.636266154806954%2C%22south%22%3A37.18248513011958%2C%22east%22%3A-119.24884988085937%2C%22west%22%3A-119.97669411914062%7D%2C%22filterState%22%3A%7B%22sort%22%3A%7B%22value%22%3A%22globalrelevanceex%22%7D%7D%2C%22isListVisible%22%3Atrue%2C%22regionSelection%22%3A%5B%7B%22regionId%22%3A13070%2C%22regionType%22%3A6%7D%5D%2C%22pagination%22%3A%7B%22currentPage%22%3A{page_num}%7D%7D"
    
    ans = get_links(driver, url, ans)
    
    driver.quit()
    time.sleep(random.uniform(5, 15))  # Random delay between 5 and 15 seconds

# Print or process the collected links
print(ans)


No more pages or error navigating to next page: Message: no such element: Unable to locate element: {"method":"css selector","selector":".a-last"}
  (Session info: chrome=125.0.6422.114); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	GetHandleVerifier [0x00ADB8E3+45827]
	(No symbol) [0x00A6DCC4]
	(No symbol) [0x0096150F]
	(No symbol) [0x009A20BC]
	(No symbol) [0x009A216B]
	(No symbol) [0x009DE0F2]
	(No symbol) [0x009C2E44]
	(No symbol) [0x009DC034]
	(No symbol) [0x009C2B96]
	(No symbol) [0x00996998]
	(No symbol) [0x0099751D]
	GetHandleVerifier [0x00D94513+2899763]
	GetHandleVerifier [0x00DE793D+3240797]
	GetHandleVerifier [0x00B613B4+593364]
	GetHandleVerifier [0x00B682DC+621820]
	(No symbol) [0x00A770A4]
	(No symbol) [0x00A737A8]
	(No symbol) [0x00A73947]
	(No symbol) [0x00A659FE]
	BaseThreadInitThunk [0x762A7BA9+25]
	RtlInitializeExceptionChain [0x7738BE3B+107]
	RtlClearBit

In [ ]:
# options = Options()
# options.add_argument('--start-maximized')
# driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()), options=options)
# ans=[]
# for page_num in range (1,5):
#     url=url=f"https://www.zillow.com/{location}/{page_num}_p/?searchQueryState=%7B%22isMapVisible%22%3Atrue%2C%22mapBounds%22%3A%7B%22north%22%3A37.636266154806954%2C%22south%22%3A37.18248513011958%2C%22east%22%3A-119.24884988085937%2C%22west%22%3A-119.97669411914062%7D%2C%22filterState%22%3A%7B%22sort%22%3A%7B%22value%22%3A%22globalrelevanceex%22%7D%7D%2C%22isListVisible%22%3Atrue%2C%22regionSelection%22%3A%5B%7B%22regionId%22%3A13070%2C%22regionType%22%3A6%7D%5D%2C%22pagination%22%3A%7B%22currentPage%22%3A{page_num}%7D%7D"
#     ans=get_links(url,ans)
#     time.sleep(random.uniform(5, 15))
# driver.quit()
    

In [13]:
for j in range (0,len(ans)):
    print(ans[j])

In [6]:
# ans=get_links(url)

In [14]:
for j in range (0,len(ans)):
    print(ans[j])

In [8]:
df = pd.DataFrame(columns=["Link",'Address', 'Price', 'Beds', 'Baths', 'Sqft', 'Property Type', 'Built Year', 'Lot Size', 'Zestimate', 'Price per sqft', 'HOA'])
# df=pd.read_csv("extracted_data.csv")

In [9]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service as ChromeService
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
import time
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service as ChromeService
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager

    # Configure Selenium to use Chrome
def get_data_for_each_url(url,df):
    # Wait for manual CAPTCHA solving
    # input("Solve the CAPTCHA and then press Enter to continue...")

    # driver.execute_script("document.body.click();")
    
    driver.get(url)

    wait = WebDriverWait(driver, 10)
    try:
        price_element = driver.find_element(By.CSS_SELECTOR, 'span[data-testid="price"] > span')
        price = price_element.text

#         container = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, 'div[data-testid="bed-bath-sqft-facts"]')))
#         beds_element = container.find_element(By.CSS_SELECTOR, 'div:nth-child(1) span.styles__StyledValueText-fshdp-8-100-2__sc-12ivusx-1.dckbUy')
#         beds = beds_element.text

#         baths_element = container.find_element(By.CSS_SELECTOR, 'button div:nth-child(1) span.styles__StyledValueText-fshdp-8-100-2__sc-12ivusx-1.dckbUy')
#         baths = baths_element.text

#         sqft_element = container.find_element(By.CSS_SELECTOR, 'div:nth-child(3) span.styles__StyledValueText-fshdp-8-100-2__sc-12ivusx-1.dckbUy')
#         sqft = sqft_element.text.replace(',', '')  # Remove commas from the number

        address_element = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, 'div.styles__AddressWrapper-fshdp-8-100-2__sc-13x5vko-0.jrtioM h1.Text-c11n-8-100-2__sc-aiai24-0.bSfDch')))
        address = address_element.text

        container = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, 'div.styles__StyledContainer-fshdp-8-100-2__sc-1dg6897-0.kcflfK')))

        # Extract the information from the container
        info_elements = container.find_elements(By.CSS_SELECTOR, 'div.styles__StyledContainer-fshdp-8-100-2__sc-6k0go5-0.kPhJKv')

        # Initialize an empty list to store extracted information
        info = []

        # Loop through the information elements and extract text
        for element in info_elements:
            info_text = element.find_element(By.CSS_SELECTOR, 'span.Text-c11n-8-100-2__sc-aiai24-0.styles__StyledFactValue-fshdp-8-100-2__sc-6k0go5-3.bSfDch.lazAga').text
            info.append(info_text)
        i1,i2,i3,i4,i5,i6=0,0,0,0,0,0
        for j in range (0,len(info)):
            if "Residential" in info[j]:
                i1=j
            elif "Built in" in info[j]:
                i2=j
            elif "lot" in info[j]:
                i3=j
            elif "Zestimate" in info[j]:
                i4=j
            elif "sqft" in info[j]:
                i5=j
            elif "HOA" in info[j]:
                i6=j
        data = {
        "Link":[url],
        "Address": [address],
        'Price': [price],
#         'Beds': [beds],
#         'Baths': [baths],
#         'Sqft': [sqft],
        'Property Type': [info[i1]],
        'Built Year': [info[i2]],
        'Lot Size': [info[i3]],
        'Zestimate': [info[i4]],
        'Price per sqft': [info[i5]],
        'HOA': [info[i6]]
        }
        new_df = pd.DataFrame(data)

    # Add the new data to the empty DataFrame by appending rows
        df = pd.concat([df, new_df], ignore_index=True)
    except Exception as e:
        print(f"Error occurred: {e}")
    
    # Close the browser

    return df




In [10]:
options = Options()
options.add_argument('--start-maximized')
driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()), options=options)
location="oakhurst-ca"
# url=f"https://www.zillow.com/homes/for_sale/{page_num}_p/?searchQueryState=%7B%22pagination%22%3A%7B%22currentPage%22%3A{page_num}%7D%2C%22isMapVisible%22%3Atrue%2C%22mapBounds%22%3A%7B%22west%22%3A-86.54695855957031%2C%22east%22%3A-86.18303644042969%2C%22south%22%3A39.465661316236485%2C%22north%22%3A39.68583094486843%7D%2C%22mapZoom%22%3A11%2C%22filterState%22%3A%7B%22sort%22%3A%7B%22value%22%3A%22globalrelevanceex%22%7D%2C%22ah%22%3A%7B%22value%22%3Atrue%7D%7D%2C%22isListVisible%22%3Atrue%7D"
for j in range (0,len(ans)):
    if df["Link"].str.contains(ans[j]).any():
        continue
    df=get_data_for_each_url(ans[j],df)
driver.quit()
df.to_csv("extracted_data.csv")

Error occurred: Message: no such element: Unable to locate element: {"method":"css selector","selector":"span[data-testid="price"] > span"}
  (Session info: chrome=125.0.6422.113); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	GetHandleVerifier [0x00FFB8E3+45827]
	(No symbol) [0x00F8DCC4]
	(No symbol) [0x00E8150F]
	(No symbol) [0x00EC20BC]
	(No symbol) [0x00EC216B]
	(No symbol) [0x00EFE0F2]
	(No symbol) [0x00EE2E44]
	(No symbol) [0x00EFC034]
	(No symbol) [0x00EE2B96]
	(No symbol) [0x00EB6998]
	(No symbol) [0x00EB751D]
	GetHandleVerifier [0x012B4513+2899763]
	GetHandleVerifier [0x0130793D+3240797]
	GetHandleVerifier [0x010813B4+593364]
	GetHandleVerifier [0x010882DC+621820]
	(No symbol) [0x00F970A4]
	(No symbol) [0x00F937A8]
	(No symbol) [0x00F93947]
	(No symbol) [0x00F859FE]
	BaseThreadInitThunk [0x762A7BA9+25]
	RtlInitializeExceptionChain [0x7738BE3B+107]
	RtlClearBits [0x77

In [11]:
df

,Link,Address,Price,Beds,Baths,Sqft,Property Type,Built Year,Lot Size,Zestimate,Price per sqft,HOA
